In [1]:
import pandas as pd
import json
import re
import csv
import pyTigerGraph as tg

# Notebook to explore and generate a graph for the Domain Record datasets

* Mangle data to make loading into the graph easier
* Instantiate the graph schema and graph queries (using TigerGraph - www.tigergraph.com )
* Perform community detection
* Collect community statistics to be used as features


In [2]:
# benign_whois_data.txt

!head "../data/raw/benign_whois_data.txt" 1


==> ../data/raw/benign_whois_data.txt <==
{"google.com": {"address": null, "city": null, "country": "US", "creation_date": [ "1997-09-15 04:00:00", "1997-09-15 07:00:00"    ], "dnssec": "unsigned", "domain_name": [ "GOOGLE.COM", "google.com"    ], "emails": [ "abusecomplaints@markmonitor.com", "whoisrequest@markmonitor.com"    ], "expiration_date": [ "2028-09-14 04:00:00", "2028-09-13 07:00:00"    ], "name": null, "name_servers": [ "NS1.GOOGLE.COM", "NS2.GOOGLE.COM", "NS3.GOOGLE.COM", "NS4.GOOGLE.COM", "ns1.google.com", "ns3.google.com", "ns4.google.com", "ns2.google.com"    ], "org": "Google LLC", "referral_url": null, "registrar": "MarkMonitor, Inc.", "state": "CA", "status": [ "clientDeleteProhibited https://icann.org/epp#clientDeleteProhibited", "clientTransferProhibited https://icann.org/epp#clientTransferProhibited", "clientUpdateProhibited https://icann.org/epp#clientUpdateProhibited", "serverDeleteProhibited https://icann.org/epp#serverDeleteProhibited", "serverTransferProhibit

In [3]:
import pandas as pd
import json

b = '../data/raw/benign_whois_data.txt'

records = []
error_lines = []

# load the lines into a list, there are some error lines that needs to be dealt with
with open(b) as f: # open file
    for line in f.read().splitlines(): # for each line
        try:
            obj = json.loads(re.sub('.*}{', '{', line)) # load the line, which looks like: {"domain.name": {...values...}}
            records.append(obj)
        except:
            error_lines.append(line)
        
print(f"Number of line errors: {len(error_lines)}")  
print(f"Number of lines: {len(records)}")     


Number of line errors: 0
Number of lines: 3731


In [4]:
record_dict = {}
for item in records:
    name = [x for x in item.keys()][0]
    record_dict[name] = item[name]

In [5]:
error_list = []
for x in record_dict:
    if type(record_dict[x]) != dict:
        error_list.append(x)
        
for x in error_list:
    del record_dict[x]

In [6]:
benign_df = pd.DataFrame.from_dict(record_dict, orient='index')
benign_df.reset_index(inplace=True)

In [7]:
benign_df.head(3)

,index,address,city,country,creation_date,dnssec,domain_name,emails,expiration_date,name,...,billing_organization,billing_phone,admin_country_code,registrar_country_code,registrar_organization_id,registrar_street,registrant_handle,registrant_zip_code,billing_street,registrant_country_code
0,google.com,None,None,US,"[1997-09-15 04:00:00, 1997-09-15 07:00:00]",unsigned,"[GOOGLE.COM, google.com]","[abusecomplaints@markmonitor.com, whoisrequest...","[2028-09-14 04:00:00, 2028-09-13 07:00:00]",None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,facebook.com,1601 Willow Rd,Menlo Park,US,1997-03-29 05:00:00,unsigned,FACEBOOK.COM,"[abusecomplaints@registrarsafe.com, domain@fb....",2031-03-30 04:00:00,Domain Admin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,youtube.com,None,None,US,2005-02-15 05:13:12,unsigned,"[YOUTUBE.COM, youtube.com]","[abusecomplaints@markmonitor.com, whoisrequest...","[2023-02-15 05:13:12, 2023-02-15 00:00:00]",None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
pd.options.display.max_rows = 4000
print(str(list(benign_df.columns)))

['index', 'address', 'city', 'country', 'creation_date', 'dnssec', 'domain_name', 'emails', 'expiration_date', 'name', 'name_servers', 'org', 'referral_url', 'registrar', 'state', 'status', 'updated_date', 'whois_server', 'zipcode', 'email', 'fax', 'phone', 'tech_email', 'tech_fax', 'tech_name', 'tech_org', 'tech_phone', 'domain__id', 'registrant_country', 'registrant_name', 'registrant_state_province', 'registrar_id', 'registrar_url', 'admin', 'admin_application_purpose', 'admin_city', 'admin_country', 'admin_email', 'admin_fax', 'admin_id', 'admin_nexus_category', 'admin_organization', 'admin_phone', 'admin_postal_code', 'admin_state_province', 'admin_street', 'registrant_application_purpose', 'registrant_city', 'registrant_email', 'registrant_fax', 'registrant_id', 'registrant_nexus_category', 'registrant_organization', 'registrant_phone', 'registrant_postal_code', 'registrant_street', 'registrar_email', 'registrar_phone', 'tech_application_purpose', 'tech_city', 'tech_country', 'te

#############

In [9]:
import pandas as pd
import json

b = '../data/raw/malicious_whois_data.txt'

records = []
error_lines = []

# load the lines into a list, there are some error lines that needs to be dealt with
with open(b) as f: # open file
    for line in f.read().splitlines(): # for each line
        try:
            obj = json.loads(re.sub('.*}{', '{', line)) # load the line, which looks like: {"domain.name": {...values...}}
            records.append(obj)
        except:
            error_lines.append(line)
        
print(f"Number of line errors: {len(error_lines)}")  
print(f"Number of lines: {len(records)}")     


Number of line errors: 0
Number of lines: 2415


In [10]:
record_dict = {}
for item in records:
    name = [x for x in item.keys()][0]
    record_dict[name] = item[name]

In [11]:
error_list = []
for x in record_dict:
    if type(record_dict[x]) != dict:
        error_list.append(x)
        
for x in error_list:
    del record_dict[x]

In [12]:
mal_df = pd.DataFrame.from_dict(record_dict, orient='index')
mal_df.reset_index(inplace=True)

In [13]:
mal_df.head(3)

,index,address,city,country,creation_date,dnssec,domain_name,emails,expiration_date,name,...,tech_phone_ext,registrar_zip_code,registrant_type,admin_account_name,admin_company_name,admin_family_name,registrant_company_name,tech_account_name,tech_company_name,tech_family_name
0,reseptors.com,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,KN,"[2021-12-08 08:53:26, 2021-12-08T08:53:26]",unsigned,RESEPTORS.COM,domainabuse@tucows.com,"[2022-12-08 08:53:26, 2022-12-08T08:53:26]",REDACTED FOR PRIVACY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,contentcdns.net,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,KN,"[2022-02-17 23:18:15, 2022-02-17T23:18:15]",unsigned,CONTENTCDNS.NET,domainabuse@tucows.com,"[2023-02-17 23:18:15, 2023-02-17T23:18:15]",REDACTED FOR PRIVACY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,izocab.com,"220013, Belarus, Minsk, ul.YA.Kolasa, d.31, kv...",Minsk,BY,2015-12-30 00:30:35,"[unsigned, Unsigned]",IZOCAB.COM,"[abuse@reg.ru, info@brale.ru]",2022-12-30 00:30:35,Vladimir Nikolskii,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#########

In [14]:
mal_df.columns 

Index(['index', 'address', 'city', 'country', 'creation_date', 'dnssec',
       'domain_name', 'emails', 'expiration_date', 'name',
       ...
       'tech_phone_ext', 'registrar_zip_code', 'registrant_type',
       'admin_account_name', 'admin_company_name', 'admin_family_name',
       'registrant_company_name', 'tech_account_name', 'tech_company_name',
       'tech_family_name'],
      dtype='object', length=119)

In [15]:
benign_df.columns

Index(['index', 'address', 'city', 'country', 'creation_date', 'dnssec',
       'domain_name', 'emails', 'expiration_date', 'name',
       ...
       'billing_organization', 'billing_phone', 'admin_country_code',
       'registrar_country_code', 'registrar_organization_id',
       'registrar_street', 'registrant_handle', 'registrant_zip_code',
       'billing_street', 'registrant_country_code'],
      dtype='object', length=154)

In [16]:
mal_df['malicious'] = 1
benign_df['malicious'] = 0

In [17]:
combined_df = pd.concat([mal_df,benign_df], axis=0)

In [18]:
len(combined_df['index'])

6062

In [19]:
DomainRecord_cols = ['index', 'dnssec', 'name', 'malicious']

combined_df.loc[:,DomainRecord_cols].to_csv("../data/external/combined_whois_data.csv")

In [20]:
list_of_explode_columns = ['country', 'emails','whois_server', 'domain_status', 'registrar', 
                           'name_servers']

In [21]:
for _col in list_of_explode_columns:
    combined_df.loc[:,['index',_col]].explode(_col).to_csv(
    f"../data/external/combined_whois_data_{_col}.csv")

In [22]:
### Entropy data import

In [23]:
# Read in the file
with open('../data/raw/benign_entropy_data.txt', 'r') as file :
  filedata = file.read()

# Replace the target string
filedata = filedata.replace('{', '')
filedata = filedata.replace('}', '')
filedata = filedata.replace(':', ',')

# Write the file out again
with open('../data/external/benign_entropy_data.txt', 'w') as file:
  file.write(filedata)

In [24]:
# Read in the file
with open('../data/raw/malicious_entropy_data.txt', 'r') as file :
  filedata = file.read()

# Replace the target string
filedata = filedata.replace('{', '')
filedata = filedata.replace('}', '')
filedata = filedata.replace(':', ',')

# Write the file out again
with open('../data/external/malicious_entropy_data.txt', 'w') as file:
  file.write(filedata)

In [25]:
### Load IP Address Org details

import pandas as pd
import json

b = '../data/raw/benign_ip_data.txt'

records = []
error_lines = []

# load the lines into a list, there are some error lines that needs to be dealt with
with open(b) as f: # open file
    for line in f.read().splitlines(): # for each line
        try:
            obj = json.loads(re.sub('.*}{', '{', line)) # load the line, which looks like: {"domain.name": {...values...}}
            records.append(obj)
        except:
            error_lines.append(line)
        
print(f"Number of line errors: {len(error_lines)}")  
print(f"Number of lines: {len(records)}")

Number of line errors: 0
Number of lines: 3731


In [26]:
b = '../data/raw/malicious_ip_data.txt'

# load the lines into a list, there are some error lines that needs to be dealt with
with open(b) as f: # open file
    for line in f.read().splitlines(): # for each line
        try:
            obj = json.loads(re.sub('.*}{', '{', line)) # load the line, which looks like: {"domain.name": {...values...}}
            records.append(obj)
        except:
            error_lines.append(line)
        
print(f"Number of line errors: {len(error_lines)}")  
print(f"Number of lines: {len(records)}")

Number of line errors: 0
Number of lines: 6146


In [27]:
record_dict = {}
for item in records:
    name = [x for x in item.keys()][0]
    record_dict[name] = item[name]

In [28]:
error_list = []
for x in record_dict:
    if type(record_dict[x]) != dict:
        error_list.append(x)
        
for x in error_list:
    del record_dict[x]

In [29]:
org_df = pd.DataFrame.from_dict(record_dict, orient='index')
org_df.reset_index(inplace=True)

In [30]:
org_df['A_Org'] = org_df['A'].apply(pd.Series)['Org']
org_df['MX_Org'] = org_df['MX'].apply(pd.Series)['Org']

In [31]:
org_df.sample(10)

,index,A,MX,A_Org,MX_Org
399,gotowebinar.com,"{'CC': 'US', 'Org': 'Microsoft Corporation'}","{'CC': 'US', 'Org': 'Proofpoint, Inc.'}",Microsoft Corporation,"Proofpoint, Inc."
5808,thousakilor.top,"{'CC': 'NA', 'Org': 'NA'}","{'CC': 'NA', 'Org': 'NA'}",NA,NA
5793,menkitostbig.top,"{'CC': 'NA', 'Org': 'NA'}","{'CC': 'NA', 'Org': 'NA'}",NA,NA
3775,acerthk3v9fvsby5n.today,"{'CC': 'US', 'Org': 'Linode'}","{'CC': 'NA', 'Org': 'NA'}",Linode,NA
4535,seztrehjplk.in,"{'CC': 'US', 'Org': 'Linode'}","{'CC': 'NA', 'Org': 'NA'}",Linode,NA
4244,bunmud42.top,"{'CC': 'NA', 'Org': 'NA'}","{'CC': 'NA', 'Org': 'NA'}",NA,NA
5539,keholus.com,"{'CC': 'NA', 'Org': 'NA'}","{'CC': 'NA', 'Org': 'NA'}",NA,NA
5013,www.blinbins.com,"{'CC': 'NA', 'Org': 'NA'}","{'CC': 'NA', 'Org': 'NA'}",NA,NA
4412,patelboostg.com,"{'CC': 'FR', 'Org': 'OVH GmbH'}","{'CC': 'NA', 'Org': 'NA'}",OVH GmbH,NA
531,billetterie.trainardeche.fr,"{'CC': 'DE', 'Org': 'Wilhelm Zwalina'}","{'CC': 'NA', 'Org': 'NA'}",Wilhelm Zwalina,NA


In [32]:
org_df.loc[org_df['MX_Org']!="NA",["index","MX_Org"]].to_csv("../data/external/combined_mx_org.csv")
org_df.loc[org_df['A_Org']!="NA",["index","A_Org"]].to_csv("../data/external/combined_A_org.csv")

In [33]:
!docker ps

CONTAINER ID   IMAGE                                     COMMAND                  CREATED       STATUS      PORTS                                                                                                                               NAMES
38e46715231e   docker.tigergraph.com/tigergraph:latest   "/bin/sh -c '/usr/sb…"   2 weeks ago   Up 2 days   0.0.0.0:9000->9000/tcp, :::9000->9000/tcp, 0.0.0.0:14240->14240/tcp, :::14240->14240/tcp, 0.0.0.0:14022->22/tcp, :::14022->22/tcp   tigergraph


In [34]:
# To start tigergraph docker
# https://docs.tigergraph.com/tigergraph-server/current/getting-started/docker
#  docker run -d -p 14022:22 -p 9000:9000 -p 14240:14240 --name tigergraph --ulimit nofile=1000000:1000000 -v /Users/nb311848/Documents/mystuff/repos/artemis/data/external:/home/tigergraph/data -t docker.tigergraph.com/tigergraph:latest


In [35]:
conn = tg.TigerGraphConnection()

In [36]:
# print(conn.gsql('ls', options=[]))

In [37]:
# Clear the server as we want to reproduce teh graph creation from scratch

print(conn.gsql('drop all', options=[]))

Dropping all, about 1 minute ...
Abort all active loading jobs
Try to abort all loading jobs on graph Artemis, it may take a while ...
[ABORT_SUCCESS] No active Loading Job to abort.
Resetting GPE...
Successfully reset GPE and GSE
Stopping GPE GSE
Successfully stopped GPE GSE in 0.004 seconds
Clearing graph store...
Successfully cleared graph store
Starting GPE GSE RESTPP
Successfully started GPE GSE RESTPP in 0.112 seconds
Everything is dropped.


In [38]:
#open text file in read mode
text_file = open("../src/scripts/DBImportExport_Artemis.gsql", "r")
 
#read whole file to a string
artemis_graph_gsql = text_file.read()
 
#close file
text_file.close()
 
# print(artemis_graph_gsql)

In [39]:
print(conn.gsql(artemis_graph_gsql, options=[]))

Stopping GPE GSE RESTPP
Successfully stopped GPE GSE RESTPP in 30.403 seconds
Starting GPE GSE RESTPP
Successfully started GPE GSE RESTPP in 0.084 seconds
The graph Artemis is created.
Successfully created schema change jobs: [create_artemis_graph].

Current graph version 0
Trying to add vertex DomainRecord.
Trying to add vertex DomainName.
Trying to add vertex Country.
Trying to add vertex City.
Trying to add vertex Emails.
Trying to add vertex Organisation.
Trying to add vertex Nameserver.
Trying to add vertex Registrar.
Trying to add vertex DomainRecordStatus.
Trying to add vertex WhoisServer.
Trying to add edge DomainRecord_DomainName.
Trying to add edge Country_City.
Trying to add edge DomainRecord_Country.
Trying to add edge DomainRecord_Emails.
Trying to add edge DomainRecord_Organisation.
Trying to add edge DomainRecord_Nameserver.
Trying to add edge DomainRecord_Registrar.
Trying to add edge DomainRecord_DomainRecordStatus.
Trying to add edge DomainRecord_WhoisServer.
Trying t

In [40]:
loading_job = """RUN LOADING JOB load_job_whois_data{field} USING MyDataSource="/home/tigergraph/data/combined_whois_data{field}.csv\""""

In [41]:
print(conn.gsql(loading_job.format(field=""),graphname='Artemis', options=[]))

for x in list_of_explode_columns:
    print(conn.gsql(loading_job.format(field=f"_{x}"),graphname='Artemis', options=[]))                       




[Tip: Use "CTRL + C" to stop displaying the loading status update, then use "SHOW LOADING STATUS jobid" to track the loading progress again]
[Tip: Manage loading jobs with "ABORT/RESUME LOADING JOB jobid"]
Starting the following job, i.e.
JobName: load_job_whois_data, jobid: Artemis.load_job_whois_data.file.m1.1649311913738
Loading log: '/home/tigergraph/tigergraph/log/restpp/restpp_loader_logs/Artemis/Artemis.load_job_whois_data.file.m1.1649311913738.log'

Job "Artemis.load_job_whois_data.file.m1.1649311913738" loading status
[WAITING] m1 ( Finished: 0 / Total: 0 )
Job "Artemis.load_job_whois_data.file.m1.1649311913738" loading status
[FINISHED] m1 ( Finished: 1 / Total: 1 )
[LOADED]
+-----------------------------------------------------------------------------------------+
|                                     FILENAME |   LOADED LINES |   AVG SPEED |   DURATION|
|/home/tigergraph/data/combined_whois_data.csv |           6063 |     29 kl/s |     0.21 s|
+--------------------------

In [42]:
# Load entropy data
loading_job = 'RUN LOADING JOB load_job_entropy USING MyDataSource="/home/tigergraph/data/benign_entropy_data.txt"'
print(conn.gsql(loading_job, graphname='Artemis', options=[]))

loading_job = 'RUN LOADING JOB load_job_entropy USING MyDataSource="/home/tigergraph/data/malicious_entropy_data.txt"'
print(conn.gsql(loading_job, graphname='Artemis', options=[]))




[Tip: Use "CTRL + C" to stop displaying the loading status update, then use "SHOW LOADING STATUS jobid" to track the loading progress again]
[Tip: Manage loading jobs with "ABORT/RESUME LOADING JOB jobid"]
Starting the following job, i.e.
JobName: load_job_entropy, jobid: Artemis.load_job_entropy.file.m1.1649311951104
Loading log: '/home/tigergraph/tigergraph/log/restpp/restpp_loader_logs/Artemis/Artemis.load_job_entropy.file.m1.1649311951104.log'

Job "Artemis.load_job_entropy.file.m1.1649311951104" loading status
[WAITING] m1 ( Finished: 0 / Total: 0 )
Job "Artemis.load_job_entropy.file.m1.1649311951104" loading status
[FINISHED] m1 ( Finished: 1 / Total: 1 )
[LOADED]
+-----------------------------------------------------------------------------------------+
|                                     FILENAME |   LOADED LINES |   AVG SPEED |   DURATION|
|/home/tigergraph/data/benign_entropy_data.txt |           3731 |     36 kl/s |     0.10 s|
+-----------------------------------------

In [43]:
# Load MX / A org data
loading_job = 'RUN LOADING JOB load_job_A_org USING MyDataSource="/home/tigergraph/data/combined_A_org.csv"'
print(conn.gsql(loading_job, graphname='Artemis', options=[]))

loading_job = 'RUN LOADING JOB load_job_MX_org USING MyDataSource="/home/tigergraph/data/combined_MX_org.csv"'
print(conn.gsql(loading_job, graphname='Artemis', options=[]))




[Tip: Use "CTRL + C" to stop displaying the loading status update, then use "SHOW LOADING STATUS jobid" to track the loading progress again]
[Tip: Manage loading jobs with "ABORT/RESUME LOADING JOB jobid"]
Starting the following job, i.e.
JobName: load_job_A_org, jobid: Artemis.load_job_A_org.file.m1.1649311961795
Loading log: '/home/tigergraph/tigergraph/log/restpp/restpp_loader_logs/Artemis/Artemis.load_job_A_org.file.m1.1649311961795.log'

Job "Artemis.load_job_A_org.file.m1.1649311961795" loading status
[WAITING] m1 ( Finished: 0 / Total: 0 )
Job "Artemis.load_job_A_org.file.m1.1649311961795" loading status
[FINISHED] m1 ( Finished: 1 / Total: 1 )
[LOADED]
+------------------------------------------------------------------------------------+
|                                FILENAME |   LOADED LINES |   AVG SPEED |   DURATION|
|/home/tigergraph/data/combined_A_org.csv |           4435 |     21 kl/s |     0.20 s|
+------------------------------------------------------------------

In [44]:
# Install all queries

print(conn.gsql('INSTALL QUERY ALL', options=[]))

Start installing queries, about 1 minute ...
delete_co_edges query: curl -X GET 'http://127.0.0.1:9000/query/Artemis/delete_co_edges'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.
tg_louvain query: curl -X GET 'http://127.0.0.1:9000/query/Artemis/tg_louvain?v_type=VALUE&e_type=VALUE&[wt_attr=VALUE]&[max_iter=VALUE]&[result_attr=VALUE]&[file_path=VALUE]&[print_info=VALUE]'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.
community_stuff query: curl -X GET 'http://127.0.0.1:9000/query/Artemis/community_stuff'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.
tg_label_prop query: curl -X GET 'http://127.0.0.1:9000/query/Artemis/tg_label_prop?v_type=VALUE&e_type=VALUE&max_iter=VALUE&output_limit=VALUE&[print_accum=VALUE]&[file_path=VALUE]&[attr=VALUE]'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.
community_features_calc query: curl -X GET 'http://127.0.0.1:9000/query/Artemis/community_features_calc'. Add -H "

In [45]:
conn.graphname = 'Artemis'

In [46]:
# Run co_edge creation

print(conn.runInstalledQuery("community_stuff"))
print(conn.runInstalledQuery("delete_co_loop_edges"))

[]
[]


In [47]:
# run label propogation

# tg_label_prop (SET<STRING> v_type, SET<STRING> e_type, INT max_iter, INT output_limit, 
#  BOOL print_accum = TRUE, STRING file_path = "", STRING attr = "") 

# have to use query strings until pyTigerGrpah allows lists for sets

params = "v_type=DomainRecord&e_type=co_registrar&e_type=c_org&e_type=co_nameserver&max_iter=10000&output_limit=0&print_accum=1&attr=community"

result = conn.runInstalledQuery("tg_label_prop", params=params)

In [49]:
result = conn.runInstalledQuery("community_features_calc")
community_features = conn.runInstalledQuery("community_features_calc")


In [82]:
community_features[0]['(@@group_entropy_final)']

[{'community': 19922987,
  'min_entropy': 2.32193,
  'max_entropy': 3.16992,
  'avg_entropy': 2.69227,
  'domain_count': 3,
  'malicious_ratio': 0},
 {'community': 22020158,
  'min_entropy': 2.9183,
  'max_entropy': 3.54659,
  'avg_entropy': 3.23244,
  'domain_count': 2,
  'malicious_ratio': 0},
 {'community': 10485769,
  'min_entropy': 2.58496,
  'max_entropy': 3.4183,
  'avg_entropy': 3.00052,
  'domain_count': 5,
  'malicious_ratio': 0},
 {'community': 24117249,
  'min_entropy': 1.5,
  'max_entropy': 1.58496,
  'avg_entropy': 1.54248,
  'domain_count': 2,
  'malicious_ratio': 0},
 {'community': 3145728,
  'min_entropy': 1.58496,
  'max_entropy': 2.32193,
  'avg_entropy': 1.9563,
  'domain_count': 4,
  'malicious_ratio': 0},
 {'community': 12582954,
  'min_entropy': 1,
  'max_entropy': 2,
  'avg_entropy': 1.5,
  'domain_count': 3,
  'malicious_ratio': 0},
 {'community': 24117392,
  'min_entropy': 1.52193,
  'max_entropy': 2.52164,
  'avg_entropy': 2.01452,
  'domain_count': 3,
  'mal

In [85]:
community_features_df = pd.DataFrame(community_features[0]['(@@group_entropy_final)'])

In [86]:
community_features_df.sample(3)

,community,min_entropy,max_entropy,avg_entropy,domain_count,malicious_ratio
34,13631495,0.91830,3.62500,2.56829,87,0.02299
87,6291458,0.00000,2.50000,1.47277,3,0.00000
52,22020201,2.25163,3.85217,3.18784,158,0.96835


In [52]:
DomainRecordsGSQL = """INTERPRET QUERY () FOR GRAPH Artemis {
   t = select dr from DomainRecord:dr;
   print(t);
}"""

domain_records = conn.runInterpretedQuery(DomainRecordsGSQL)


In [59]:
domain_records[0]['(t)']

[{'v_id': 'google.ee',
  'v_type': 'DomainRecord',
  'attributes': {'id': 'google.ee',
   'dnssec': '',
   'name': '',
   'entropy': 1.9183,
   'MaliciousFlag': False,
   'community': 9437303}},
 {'v_id': 'aftonbladet.se',
  'v_type': 'DomainRecord',
  'attributes': {'id': 'aftonbladet.se',
   'dnssec': 'unsigned delegation',
   'name': '',
   'entropy': 3.0958,
   'MaliciousFlag': False,
   'community': -1}},
 {'v_id': 'csiro.au',
  'v_type': 'DomainRecord',
  'attributes': {'id': 'csiro.au',
   'dnssec': '',
   'name': '',
   'entropy': 2.32193,
   'MaliciousFlag': False,
   'community': -1}},
 {'v_id': 'google.sk',
  'v_type': 'DomainRecord',
  'attributes': {'id': 'google.sk',
   'dnssec': '',
   'name': '',
   'entropy': 1.9183,
   'MaliciousFlag': False,
   'community': 9437303}},
 {'v_id': 'data.gov.uk',
  'v_type': 'DomainRecord',
  'attributes': {'id': 'data.gov.uk',
   'dnssec': '',
   'name': '',
   'entropy': 1.5,
   'MaliciousFlag': False,
   'community': 2097154}},
 {'v_i

In [62]:
dr_dict = {}

for x in domain_records[0]['(t)']:
    dr_dict[x['v_id']] = x['attributes']['community']

In [67]:
dr_dict

{'google.ee': 9437303,
 'aftonbladet.se': -1,
 'csiro.au': -1,
 'google.sk': 9437303,
 'data.gov.uk': 2097154,
 'dailyrecord.co.uk': 2097154,
 'bl.uk': -1,
 'manchestereveningnews.co.uk': 2097154,
 'nicovideo.jp': -1,
 'soup.io': 17825818,
 'redd.it': 1048579,
 'google.cn': 9437303,
 'kiev.ua': 2097152,
 'gouv.fr': -1,
 'thunderbird.net': 9437303,
 'datingranking.net': 31457366,
 'spokesman.com': 24117291,
 'southwest.com': 9437303,
 'postimage.org': 17825818,
 'peninsuladailynews.com': 17825818,
 'nmsu.edu': -1,
 'ox.ac.uk': 2097154,
 'timesnownews.com': 17825818,
 'apkpure.com': 9437303,
 'habr.com': 1048600,
 'googlesyndication.com': 9437303,
 'business.com': 13631495,
 'opensecrets.org': 13631495,
 'ipsos.com': 12583022,
 'tvguide.com': 20971641,
 'buffalonews.com': 17825818,
 'lufthansa.com': 22020179,
 'gazeta.ru': 8388758,
 'sxsw.com': 17825818,
 'pantone.com': 17825818,
 'akamai.com': 9437303,
 'siteground.com': 22020165,
 'smore.com': 13631495,
 'scopus.com': 17825976,
 'world

In [87]:
domain_record_df = pd.DataFrame.from_dict(dr_dict, orient='index').reset_index()
domain_record_df.columns = ['DomainRecord','community']

In [88]:
domain_record_df.sample(3)

,DomainRecord,community
5857,mi.com,24117291
1433,cba.pl,-1
642,holarty.com,1048689


In [90]:
graph_features_df = domain_record_df.merge(community_features_df, how = 'left', on='community')

In [93]:
graph_features_df.sample(10)

,DomainRecord,community,min_entropy,max_entropy,avg_entropy,domain_count,malicious_ratio
4910,wixsite.com,17825818,0.00000,3.82687,2.70943,398,0.05276
3932,bliblah.com,22020165,0.91830,3.89366,2.95632,153,0.55556
2992,viewsketplctly.fun,31457366,0.00000,3.72548,2.69222,467,0.74732
4936,generatepress.com,17825818,0.00000,3.82687,2.70943,398,0.05276
2527,uptodown.com,19923097,0.91830,3.91613,2.71832,137,0.19708
2959,laboratory.bmesarsodothmqoubieo.com,23068710,2.52164,3.96900,3.63355,56,1.00000
1229,cafe24.com,19922987,2.32193,3.16992,2.69227,3,0.00000
5626,contentcdns.net,22020165,0.91830,3.89366,2.95632,153,0.55556
1020,clarin.com,24117291,0.00000,3.66450,2.48872,286,0.00699
4607,klm.com,9437303,0.00000,4.20830,2.53622,600,0.00833


In [95]:
graph_features_df.to_csv('../data/processed/graph_community_features.csv', index=False)